# Image Processing SS 16 - Assignment - 07

### Deadline is 12.6.2016 at 23:99 o'clock

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [1]:
# display the plots inside the notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io.wavfile
from skimage.data import astronaut
from skimage.color import rgb2gray

from __future__ import division
import random
try:
    from StringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
    
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
    
    
from numpy.fft import fft2 as numpy_fft2, ifft2 as numpy_ifft2

from PIL import Image
import itertools
import IPython

pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

In [3]:
# Playground to get some intuition for 2d fourier transfromation
ft = np.zeros((64,64))

# set some random coefficents and see what happens
#
ft[0, 1] = 1

ft[10, 10] = 1

plt.subplot(121)
plt.imshow(np.real(numpy_ifft2(ft)), cmap='gray')
plt.subplot(122)
plt.imshow(np.imag(numpy_ifft2(ft)), cmap='gray')
plt.show()

# Exercise 1 - 2D DFT - 2 Points
Implement the 2D Discrete Fourier Transfromation with [Matrix Multiplication](https://en.wikipedia.org/wiki/DFT_matrix). You can use your code from the last assignment to compute the dct matrix.

In [4]:
def dft(n,s,norm):
    i, j = np.meshgrid(np.arange(n), np.arange(n))
    omega = np.exp( - 2 * np.pi * 1J / n * s)
    dft = np.power( omega, i * j ) / norm
    return dft

def dft2d(img):
    """
    Returns the 2d discrete fourier transformation
    """
    n = max(img.shape[0],img.shape[0])
    W = dft(n,1,np.sqrt(n))
    return W.dot(img).dot(W)

def inv_dft2d(x):
    """
    Returns the 2d inverse discrete fourier transformation
    """
    n = max(x.shape[0],x.shape[0])
    Wi = dft(n,-1,1)
    return Wi.dot(x).dot(Wi)


def chess_board(n=8, field_size=32):
    board = np.zeros((n*field_size, n*field_size))
    s = field_size
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i*s:(i+1)*s, j*s:(j+1)*s] = 1
    return board

plt.subplot(321)
plt.imshow(np.real(chess_board()), cmap='gray')
plt.subplot(322)
plt.imshow(np.imag(chess_board()), cmap='gray')
plt.subplot(323)
plt.imshow(np.real(dft2d(chess_board())), cmap='gray')
plt.subplot(324)
plt.imshow(np.imag(dft2d(chess_board())), cmap='gray')
plt.subplot(325)
plt.imshow(np.real(inv_dft2d(dft2d(chess_board()))), cmap='gray')
plt.subplot(326)
plt.imshow(np.imag(inv_dft2d(dft2d(chess_board()))), cmap='gray')
plt.show

# Exercise 3 - High and Low Pass filter with the 2D fourier transfromation

In [5]:
def high_pass(img_ft, n):
    """Removes the low frequencies"""
    ft = img_ft.copy()
    ft[:n, :n] = 0
    ft[-n:, :n] = 0
    ft[-n:, -n:] = 0
    ft[:n, -n:] = 0
    return ft

def low_pass(img_ft, n):
    """Removes the high frequencies"""
    ft = img_ft.copy()
    ft[:, n:-n] = 0
    ft[n:-n, :] = 0
    return ft
    
def band_pass(img_ft, low, high):
    """Only perserve the frequencies between low and high"""
    # your code here
    ft = high_pass(img_ft, low)
    ft = low_pass(ft, high)
    return ft
    
def inv_dft_and_plot(img_ft):
    plt.imshow(np.real(inv_dft2d(img_ft)), cmap='gray')
    plt.show()

In [6]:
chess_board_ft = dft2d(chess_board())

# display frequency domain of the chess_board
plt.subplot(121)
plt.imshow(np.real(chess_board_ft), cmap='gray')
plt.subplot(122)
plt.imshow(np.imag(chess_board_ft), cmap='gray')
plt.show()

In [7]:
# remove the low frequiencies of the chess board
chess_high_feq = high_pass(chess_board_ft, 12)
inv_dft_and_plot(chess_high_feq)

In [8]:
# remove the high frequiencies of the chess board
chess_low_feq = low_pass(chess_board_ft, 10)
inv_dft_and_plot(chess_low_feq)

In [9]:
# remove the high frequiencies of the chess board
chess_medium_feq = band_pass(chess_board_ft, 5, 15)
inv_dft_and_plot(chess_medium_feq)

In [10]:
img = rgb2gray(astronaut() / 255)
# filter all frequencies corresponding to the coeffientns between 48 and 128.
astro_dft = dft2d(img)
astro_high = high_pass(astro_dft,128)
astro_low = low_pass(astro_dft,48)
astro = astro_high + astro_low
inv_dft_and_plot(astro)

# Exercise 3 - FFT - 5 Points

Implement the fast fourier transformation. 
Use the [Cooley-Tukey](https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm) algorithm.

In [11]:
def fft1(x):
    N = len(x)
    if N <= 1: return x
    even = fft1(x[0::2])
    odd =  fft1(x[1::2])
    T = [np.exp(-2j*np.pi*k/N) * odd[k] for k in range(N//2)]
    return [even[k] + T[k] for k in range(N//2)] + [even[k] - T[k] for k in range(N//2)]

def fft(x):
    """
    Computes the fast fourier transformation of x.
    """
    temp = np.zeros(x.shape, dtype ='complex')
    X = np.zeros(x.shape, dtype ='complex')
    
    temp = np.apply_along_axis(fft1,1,x)
    X = np.apply_along_axis(fft1,0,temp)
    return X


In [12]:
img = rgb2gray(astronaut() / 255)
fft_img = fft(img)

In [13]:
plt.subplot(121)
plt.imshow(np.real(fft_img),cmap="gray")
plt.subplot(122)
plt.imshow(np.imag(fft_img),cmap="gray")
plt.show()

In [14]:
plt.subplot(121)
plt.imshow(np.real(inv_dft2d(fft_img)),cmap="gray")
plt.subplot(122)
plt.imshow(np.imag(inv_dft2d(fft_img)),cmap="gray")
plt.show()